# Lesson 2

Example code. Let's look at examples of how you can use a pre-built AI Connector with Semantic Kernel Python and .Net that uses auto-function calling to have the model respond to user input:

In [ ]:
# Semantic Kernel Python Example

import asyncio # library for asynchronous programming (non-blocking tasks) = improve efficiency
# normally python runs code line by line and waits for each step to finish before moving on = BLOCKING 
# async useful - when callng external services like azure oienai, program might wait for response for few seconds - if we block during that time, nothing else can run
# async solves this by letting program do other work while waiting for response 
from typing import Annotated # adds metadata to type hints (used for function argument descriptions)

from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings # connector to azure openai
from semantic_kernel.contents import ChatHistory # stores conversation context
from semantic_kernel.functions import kernel_function # decorator that marks a method (function in a class) as callable by the LLM
from semantic_kernel.kernel import Kernel # core orchestrator that manages plugins and AI connectors

# Define a ChatHistory object to hold the conversation's context
chat_history = ChatHistory()
chat_history.add_user_message("I'd like to go to New York on January 1, 2025")


# Define a sample plugin that contains the function (method) to book travel
class BookTravelPlugin:
    """A Sample Book Travel Plugin"""

    @kernel_function(name="book_flight", description="Book travel given location and date") # this decorator tells semantic kernel this function can be called by LLM (registers the method (book_flight) as a callable tool for LLM)
    async def book_flight( # async def makes this function asynchronous so it doesn't block other tasks
        self, date: Annotated[str, "The date of travel"], location: Annotated[str, "The location to travel to"] #Adds descriptions for arguments (helps the LLM know what to supply).
    ) -> str: # annotated above - LLM uses the descrpitions to know what arguments it needs to supply when calling the function
        return f"Travel was booked to {location} on {date}" # returns a confirmation string "mock" - in real app, replace with an API call to airline system  + database update + email conf.
    
    # self is always first argument in class method -- refers to instance of the class (so method can access class attributes)
    # date and location are the actual args passed by the LLM when it calls the function

# Create the Kernel
kernel = Kernel() # creates a kernel instance

# Add the sample plugin to the Kernel object
# plugin is just a class with 1+ functions that the LLM can call
kernel.add_plugin(BookTravelPlugin(), plugin_name="book_travel") # registers the plugin
# after registering, the LLM knows: plugin name (book_travel) and functions inside it (book_flight)

# not all fucntions need to be in same class - you can put multiple related functions in one class, or diff classes for diff domains
# each class becomes separate plugin when registered

# Define the Azure OpenAI AI Connector - this connects to azure openai for chat completions
chat_service = AzureChatCompletion(
    deployment_name="YOUR_DEPLOYMENT_NAME", 
    api_key="YOUR_API_KEY", 
    endpoint="https://<your-resource>.azure.openai.com/",
)

# Define the request settings to configure the model with auto-function calling
request_settings = AzureChatPromptExecutionSettings(function_choice_behavior=FunctionChoiceBehavior.Auto()) # lets model automatically decide when to call our function

async def main(): # main is entry point for async workflow
    # Make the request to the model for the given chat history and request settings
    # The Kernel contains the sample that the model will request to invoke
    response = await chat_service.get_chat_message_content( # await waits for the async call to finish "pause here until result comes back, but don't freeze everything else"
        chat_history=chat_history, settings=request_settings, kernel=kernel # passes the chat history, settings, and kernel (with plugins) to the model
    )
    
    assert response is not None

    """
    Note: In the auto function calling process, the model determines it can invoke the 
    `BookTravelPlugin` using the `book_flight` function, supplying the necessary arguments. 
    
    For example:

    "tool_calls": [
        {
            "id": "call_abc123",
            "type": "function",
            "function": {
                "name": "BookTravelPlugin-book_flight",
                "arguments": "{'location': 'New York', 'date': '2025-01-01'}"
            }
        }
    ]

    Since the location and date arguments are required (as defined by the kernel function), if the 
    model lacks either, it will prompt the user to provide them. For instance:

    User: Book me a flight to New York.
    Model: Sure, I'd love to help you book a flight. Could you please specify the date?
    User: I want to travel on January 1, 2025.
    Model: Your flight to New York on January 1, 2025, has been successfully booked. Safe travels!
    """

    print(f"`{response}`")
    # Example AI Model Response: `Your flight to New York on January 1, 2025, has been successfully booked. Safe travels! ✈️🗽`
    

    # Add the model's response to our chat history context
    chat_history.add_assistant_message(response.content)

# the above sends chat history and plugin info to model, which decides if needs to call book_flight
# model returns a response e.g. your glight is booked - which we print and add to chat history

if __name__ == "__main__": # runs the async main() function in an event loop
    asyncio.run(main())

# Summary: 
# Kernel = Orchestrator.
# Plugin = Your custom tools.
# LLM = Decides when to call tools.
# Async = Makes network calls efficient.
# Annotated = Helps LLM understand arguments.
# @kernel_function = Registers the function for auto-calling.

Example code (AutoGen):

In [ ]:
# creating agents, then create a round robin schedule where they can work together, in this case in order

# Data Retrieval Agent --> fetches data using a tool
# Data Analysis Agent --> analyses data using another tool
# Decision Making Agent --> represents the human user

# Then it sets up a Round Robin schedule so these agents can collaborate in turns until a termination condition is met (user says "APPROVE").
# Finally, it runs the conversation as a stream.

### What is Happening Conceptually

# 1. User says “Analyze data”.
# 2. Retrieve agent fetches data using retrieve_tool.
# 3. Analyze agent processes data using analyze_tool.
# 4. User proxy can approve or give feedback.
# 5. Repeat until user says "APPROVE" or max turns reached.

###Do all tools need to be in one class?
# No. Tools can be standalone functions or grouped in plugins. Agents can have multiple tools.

agent_retrieve = AssistantAgent( # AssistantAgent: A class that wraps an LLM and gives it a role.
    name="dataretrieval", # identifier for the agent
    model_client=model_client, # The LLM backend (e.g., OpenAI, Azure).
    tools=[retrieve_tool], # Functions the agent can call (here, retrieve_tool).
    system_message="Use tools to solve tasks." # Defines the agent’s behavior (like a role prompt).
)

agent_analyze = AssistantAgent(
    name="dataanalysis",
    model_client=model_client,
    tools=[analyze_tool],
    system_message="Use tools to solve tasks."
)

# conversation ends when user says "APPROVE"
termination = TextMentionTermination("APPROVE") # Conversation stops when any message contains “APPROVE”.This is useful for controlled workflows.

user_proxy = UserProxyAgent("user_proxy", input_func=input) # Represents the human user. input_func=input means it will read user input from the console.

team = RoundRobinGroupChat([agent_retrieve, agent_analyze, user_proxy], termination_condition=termination)
# above creates a team of agents
# round robin means each agent takes turns in a fixed order. e.g., retrieve --> analyse --> user --> retrieve --> analyse --> ...
# stops when termination condition is met

stream = team.run_stream(task="Analyze data", max_turns=10) # run_stream strarts the convo, initial task is analyse data, max turns is 10
# Use asyncio.run(...) when running in a script.
await Console(stream) # await because this is async (non-blocking) - async lets multiple agents work efficiently without blocking

Here you have a short code snippet in which you create your own agent with Chat capabilities:

In [ ]:
from autogen_agentchat.agents import AssistantAgent # AssistantAgent: A ready-made agent that can handle chat completions.
from autogen_agentchat.messages import TextMessage # TextMessage: Represents a message in the conversation.
from autogen_ext.models.openai import OpenAIChatCompletionClient # OpenAIChatCompletionClient: Connects to OpenAI’s GPT models.

# This code shows how to:

# 1. Create a custom agent (MyAgent) that can handle messages.
# 2. Use AutoGen’s runtime to register the agent and send messages.
# 3. Delegate actual chat handling to a pre-built AssistantAgent (which knows how to talk to an LLM like GPT-4).

# MyAgent → AssistantAgent → GPT-4 → Response


class MyAgent(RoutedAgent): # MyAgent inherits from RoutedAgent (a base class for agents that can route messages).
    def __init__(self, name: str) -> None:
        super().__init__(name) # to initialize the parent class.
        model_client = OpenAIChatCompletionClient(model="gpt-4o") # Creates model_client for GPT-4.
        self._delegate = AssistantAgent(name, model_client=model_client) # Assigns self._delegate = an AssistantAgent instance.
        # This means MyAgent doesn’t handle chat logic itself—it delegates to AssistantAgent.

    @message_handler #  A decorator that marks this method as a handler for a specific message type.
    async def handle_my_message_type(self, message: MyMessageType, ctx: MessageContext) -> None: # async def: Because message handling involves calling the LLM (network I/O).
        print(f"{self.id.type} received message: {message.content}") # Print the incoming message.
        response = await self._delegate.on_messages( # Call self._delegate.on_messages(...): Converts the message into a TextMessage. Passes it to the AssistantAgent (which talks to GPT-4).
            [TextMessage(content=message.content, source="user")], ctx.cancellation_token
        )
        print(f"{self.id.type} responded: {response.chat_message.content}") # Print the LLM’s response.

In [ ]:

# main.py
runtime = SingleThreadedAgentRuntime() # Runs agents in a single thread.Runtime Orchestrates agents and message passing.
await MyAgent.register(runtime, "my_agent", lambda: MyAgent())  # Registers your custom agent with the runtime. "my_agent" = agent name. lambda: MyAgent() = factory function to create the agent.
# Agent registration: Tells AutoGen what agents exist and how to create them.

# Here, runtime.start() and send_message() kick off the event loop for message processing
runtime.start()  # Start processing messages in the background.
await runtime.send_message(MyMessageType("Hello, World!"), AgentId("my_agent", "default")) # Sends "Hello, World!" to my_agent.

Output:
<br>my_agent received message: Hello, World!
<br>my_assistant received message: Hello, World!
<br>my_assistant responded: Hello! How can I assist you today?